In [ ]:
!pip install evaluate datasets

In [ ]:
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

In [3]:
import torch
import evaluate

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
from datasets import Dataset

In [4]:
!wget https://raw.githubusercontent.com/AlexanderNovikovITMO/NLP-ML_in_ITMO/main/CompiledjobListNigeria.csv

--2023-05-18 23:17:22--  https://raw.githubusercontent.com/AlexanderNovikovITMO/NLP-ML_in_ITMO/main/CompiledjobListNigeria.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 109355 (107K) [text/plain]
Saving to: ‘CompiledjobListNigeria.csv’

CompiledjobListNige 100%[===================>] 106.79K  --.-KB/s    in 0.002s  

2023-05-18 23:17:22 (58.9 MB/s) - ‘CompiledjobListNigeria.csv’ saved [109355/109355]



In [5]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [6]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
metric = evaluate.load("accuracy")

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [7]:
def tokenize_function(examples):
    return tokenizer(examples["job_requirement"], padding="max_length", truncation=True)

In [8]:
dataset = pd.read_csv('CompiledjobListNigeria.csv')
train, test = train_test_split(dataset[['job_requirement', 'label']], test_size=0.2, random_state=2023)
train_dataset = Dataset.from_pandas(train).class_encode_column("label").map(tokenize_function, batched=True)
test_dataset = Dataset.from_pandas(test).class_encode_column("label").map(tokenize_function, batched=True)

Stringifying the column:   0%|          | 0/161 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/161 [00:00<?, ? examples/s]

Map:   0%|          | 0/161 [00:00<?, ? examples/s]

Stringifying the column:   0%|          | 0/41 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/41 [00:00<?, ? examples/s]

Map:   0%|          | 0/41 [00:00<?, ? examples/s]

In [9]:
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

In [11]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.293448,0.902439
2,No log,0.308619,0.926829
3,No log,0.357408,0.926829


TrainOutput(global_step=63, training_loss=0.1965032002282521, metrics={'train_runtime': 50.175, 'train_samples_per_second': 9.626, 'train_steps_per_second': 1.256, 'total_flos': 127082639738880.0, 'train_loss': 0.1965032002282521, 'epoch': 3.0})